<h3>0. Exploratory analysis</h3>

In [3]:
import sys
print sys.version[0]
# sys.setdefaultencoding('utf8')
print sys.getdefaultencoding()
# reload(sys)  
# sys.setdefaultencoding('utf8')
# print "Done"

2
ascii


In [11]:
import json
import pandas as pd

df_training=pd.read_json('project_files/training.json', encoding = 'utf8')
df_devel=pd.read_json('project_files/devel.json')
df_docs=pd.read_json('project_files/documents.json')
df_testing=pd.read_json('project_files/testing.json')

print "Done"

Done


In [ ]:
for item in df_training['text'][:1]:
    print(item)

In [ ]:
df_training.head()

In [ ]:
df_training.dtypes

In [ ]:
df_docs.head()

In [ ]:
df_testing.head()

In [ ]:
df_devel.head()

<h3>1. Question processing</h3>

In [ ]:
from collections import defaultdict
import nltk

def process_question(words):
    ## question type: classify the type of question
    # hand writter rule vs supervised ML - start with rules then we can move to supervised using datasets like.
    
    # default values
    question_type='UNK'
    answer_type='UNK'
    query=''
    
    # handwriten rules
    y_n_question=['is','can','could']
    wh_question=['what','who','where','when']
    
    for word in words:
        word=word.lower()
        if word in y_n_question:
            question_type='YN'
            break
        elif word in wh_question:
            question_type='WH'
            break
        else:
            question_type='UNK'
            
    
    ## focus: strings possible to be replaced in the answer
    # identify with the NER - use NLP taggin from CoreNLP
    
    ## answer type: kind of entity
    # from NER
    
    # just to start, the answer type is the same as question -- DELETE
    answer_type=question_type
    
    ## query: keywords to be used for the IR system to search in documents
    # do we need to do semantic parsing?
    # query should be built according to question type
    
    
    
    return query,answer_type

# process questions
queries=[]
for index, row in df_training.iterrows():
    question_training=nltk.word_tokenize(row['question'])
    query,answer_type=process_question(question_training)
    queries.append((query,answer_type))
    
    

In [ ]:
queries[1:4]

NER

In [12]:
df_training.head()

,answer_paragraph,docid,question,text
0,23,0,A kilogram could be definined as having a Plan...,6966662606895999999♠6.62606896×10−34 j⋅s
1,22,0,What is the shape of the object that establish...,cylinder
2,12,0,What example is given as another paired relati...,time vs. energy
3,1,0,What does the Planck Constant refer to?,quantum of action
4,10,0,When was the first quantized model of the atom...,1913


In [19]:
import nltk
tokenized_sentences = [nltk.word_tokenize(sentence.strip()) for sentence in df_training['question']]
print "Done"

Done


In [20]:
print tokenized_sentences[0]

[u'A', u'kilogram', u'could', u'be', u'definined', u'as', u'having', u'a', u'Planck', u'constant', u'of', u'what', u'value', u'?']


In [ ]:
# nltk.download('maxent_ne_chunker')
# nltk.download('words')
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')

In [15]:
tagged_sentences = [nltk.pos_tag(sentence) for sentence in tokenized_sentences]
print tagged_sentences[0]

[(u'A', 'DT'), (u'kilogram', 'NN'), (u'could', 'MD'), (u'be', 'VB'), (u'definined', 'VBN'), (u'as', 'IN'), (u'having', 'VBG'), (u'a', 'DT'), (u'Planck', 'NNP'), (u'constant', 'NN'), (u'of', 'IN'), (u'what', 'WP'), (u'value', 'NN'), (u'?', '.')]


In [16]:
chunked_sentences = nltk.ne_chunk_sents(tagged_sentences, binary=True)
print "Done"

Done


In [17]:
def extract_entity_names(t):
    entity_names = []

    if hasattr(t, 'label') and t.label:
        if t.label() == 'NE':
            entity_names.append(' '.join([child[0] for child in t]))
        else:
            for child in t:
                entity_names.extend(extract_entity_names(child))

    return entity_names

entity_names = []
entity_name = []
for tree in chunked_sentences:
    # Print results per sentence
    print tree
#     print extract_entity_names(tree)
    entity_names.extend(extract_entity_names(tree))
    entity_name.append(extract_entity_names(tree))

# Print all entity names
#print entity_names

# Print unique entity names
# print set(entity_names)

(S
  A/DT
  kilogram/NN
  could/MD
  be/VB
  definined/VBN
  as/IN
  having/VBG
  a/DT
  Planck/NNP
  constant/NN
  of/IN
  what/WP
  value/NN
  ?/.)
(S
  What/WP
  is/VBZ
  the/DT
  shape/NN
  of/IN
  the/DT
  object/NN
  that/WDT
  establishes/VBZ
  the/DT
  base/JJ
  unit/NN
  of/IN
  the/DT
  kilogram/NN
  ?/.)
(S
  What/WP
  example/NN
  is/VBZ
  given/VBN
  as/IN
  another/DT
  paired/JJ
  relationship/NN
  of/IN
  uncertainly/RB
  related/VBN
  to/TO
  standard/VB
  deviation/NN
  ?/.)
(S
  What/WP
  does/VBZ
  the/DT
  (NE Planck/NNP Constant/NNP)
  refer/NN
  to/TO
  ?/.)
(S
  When/WRB
  was/VBD
  the/DT
  first/JJ
  quantized/JJ
  model/NN
  of/IN
  the/DT
  atom/NN
  introduced/VBN
  ?/.)
(S
  What/WP
  scientist/NN
  first/RB
  studied/VBD
  black/JJ
  body/NN
  radiation/NN
  ?/.)
(S
  Who/WP
  helped/VBD
  to/TO
  give/VB
  the/DT
  correct/JJ
  quantization/NN
  rules/NNS
  for/IN
  electrons/NNS
  in/IN
  1926/CD
  ?/.)
(S
  Classical/JJ
  statistical/JJ
  mechanics/NNS

(S
  What/WP
  involves/VBZ
  the/DT
  deployment/NN
  of/IN
  three/CD
  generations/NNS
  of/IN
  renewable/JJ
  energy/NN
  technologies/NNS
  dating/VBG
  back/RB
  more/JJR
  than/IN
  100/CD
  years/NNS
  ?/.)
(S
  Who/WP
  believes/VBZ
  that/IN
  the/DT
  market/NN
  does/VBZ
  not/RB
  value/NN
  nature/NN
  's/POS
  service/NN
  adequately/RB
  ?/.)
(S
  How/WRB
  many/JJ
  countries/NNS
  now/RB
  have/VBP
  renewable/JJ
  energy/NN
  policies/NNS
  ?/.)
(S
  (NE Renewables/NNS)
  contributed/VBD
  what/WP
  percentage/NN
  to/TO
  our/PRP$
  energy/NN
  consumption/NN
  ?/.)
(S
  What/WP
  is/VBZ
  a/DT
  promising/VBG
  cellulose/JJ
  source/NN
  that/WDT
  can/MD
  be/VB
  sustainably/RB
  produced/VBN
  in/IN
  many/JJ
  regions/NNS
  ?/.)
(S
  Some/DT
  countries/NNS
  have/VBP
  phased/VBN
  out/RP
  all/DT
  subsidies/NNS
  for/IN
  what/WP
  substance/NN
  ?/.)
(S
  What/WP
  was/VBD
  the/DT
  result/NN
  of/IN
  publicly/RB
  funded/VBN
  links/NNS
  among/IN
  com

(S
  What/WP
  job/NN
  was/VBD
  (NE SPielberg/NNP)
  offered/VBD
  when/WRB
  he/PRP
  was/VBD
  a/DT
  student/NN
  ?/.)
(S
  How/WRB
  many/JJ
  bloody/JJ
  noses/NNS
  did/VBD
  (NE Spielberg/NNP)
  get/VB
  in/IN
  (NE High/JJ School/NNP)
  ?/.)
(S
  In/IN
  what/WP
  film/NN
  did/VBD
  (NE Spielberg/NNP)
  address/NN
  humanistic/JJ
  issues/NNS
  ?/.)
(S
  What/WP
  artifact/NN
  did/VBD
  (NE Indy/NNP)
  's/POS
  father/NN
  focus/NN
  on/IN
  ?/.)
(S Who/WP starred/VBD in/IN 'Savage'//NN)
(S
  When/WRB
  did/VBD
  the/DT
  (NE Visual/NNP Effects/NNPS Society/NNP)
  give/VBP
  Spielberg/NNP
  a/DT
  Lifetime/NNP
  Achievement/NNP
  Award/NNP
  ?/.)
(S
  When/WRB
  did/VBD
  (NE Spielberg/NNP)
  and/CC
  Irving/NNP
  divorce/NN
  ?/.)
(S
  When/WRB
  did/VBD
  (NE Spielberg/NNP)
  first/RB
  play/VB
  a/DT
  video/NN
  game/NN
  ?/.)
(S (NE Which/JJ) channel/NN aired/VBD 'Terra/CD Nova/NNP '/POS ?/.)
(S
  How/WRB
  many/JJ
  (NE Academy/NNP Awards/NNPS)
  did/VBD
  'Jaws/NNP
 

(S
  What/WP
  was/VBD
  also/RB
  utilized/JJ
  to/TO
  control/VB
  ground/NN
  targets/NNS
  ?/.)
(S
  In/IN
  the/DT
  1920s/CD
  ,/,
  which/WDT
  company/NN
  developed/VBD
  the/DT
  automatic/JJ
  20/CD
  mm/NN
  ?/.)
(S
  What/WP
  war/NN
  was/VBD
  the/DT
  (NE FlaK/NNP)
  first/RB
  used/VBD
  in/IN
  ?/.)
(S
  What/WP
  was/VBD
  the/DT
  name/NN
  of/IN
  the/DT
  mechanical/JJ
  computer/NN
  that/WDT
  used/VBD
  automation/NN
  ?/.)
(S What/WP year/NN was/VBD the/DT (NE ADGB/NNP) created/VBD ?/.)
(S
  What/WP
  particular/JJ
  aircraft/NN
  did/VBD
  the/DT
  interceptor/NN
  target/NN
  most/RBS
  ?/.)
(S
  How/WRB
  many/JJ
  guns/NNS
  or/CC
  missile/NN
  launchers/NNS
  are/VBP
  typically/RB
  in/IN
  a/DT
  battery/NN
  ?/.)
(S
  What/WP
  was/VBD
  the/DT
  (NE HRF/NNP)
  used/VBD
  with/IN
  to/TO
  set/VB
  fuses/NNS
  ?/.)
(S
  What/WP
  are/VBP
  man-portable/JJ
  missiles/NNS
  better/RB
  known/VBN
  as/IN
  ?/.)
(S
  The/DT
  (NE HRF/NNP)
  measured/VBD


(S
  (NE Rocket/NNP)
  research/NN
  began/VBD
  prior/RB
  to/TO
  which/WDT
  war/NN
  in/IN
  some/DT
  countries/NNS
  ?/.)
(S
  (NE Which/JJ)
  military/NN
  set/VBD
  up/RP
  a/DT
  large/JJ
  air/NN
  defence/NN
  network/NN
  surrounding/VBG
  its/PRP$
  larger/JJR
  cities/NNS
  ?/.)
(S
  What/WP
  is/VBZ
  the/DT
  term/NN
  used/VBN
  to/TO
  describe/VB
  the/DT
  height/NN
  that/IN
  a/DT
  projectile/NN
  would/MD
  go/VB
  to/TO
  if/IN
  it/PRP
  was/VBD
  fired/VBN
  vertically/RB
  ?/.)
(S Who/WP designed/VBD this/DT cannon/NN ?/.)
(S
  The/DT
  military/NN
  was/VBD
  not/RB
  allowed/VBN
  to/TO
  use/VB
  anything/NN
  that/WDT
  was/VBD
  bigger/JJR
  than/IN
  what/WP
  ?/.)
(S
  What/WP
  was/VBD
  the/DT
  designation/NN
  name/NN
  of/IN
  the/DT
  (NE Sperry/NNP)
  device/NN
  in/IN
  the/DT
  (NE US/NNP)
  ?/.)
(S
  Another/DT
  name/NN
  for/IN
  the/DT
  (NE High/NNP Bunkers/NNP)
  was/VBD
  what/WP
  ?/.)
(S
  (NE Which/JJ)
  military/JJ
  section/NN
  c

...

(S
  How/WRB
  much/JJ
  of/IN
  a/DT
  drop/NN
  was/VBD
  there/RB
  in/IN
  manufacturing/VBG
  jobs/NNS
  between/IN
  1990/CD
  and/CC
  2010/CD
  ?/.)
(S
  How/WRB
  many/JJ
  catholic/JJ
  schools/NNS
  are/VBP
  in/IN
  (NE Paris/NNP)
  ?/.)
(S
  Who/WP
  realied/VBD
  the/DT
  first/JJ
  digital/JJ
  cinema/NN
  projection/NN
  in/IN
  (NE Europe/NNP)
  ?/.)
(S
  Who/WP
  created/VBD
  (NE Paris/NNP)
  '/POS
  reservoir/NN
  system/NN
  ?/.)
(S
  In/IN
  what/WP
  arrondissement/NN
  is/VBZ
  the/DT
  (NE Sorbonne/NNP)
  library/NN
  in/IN
  (NE Paris/NNP)
  located/VBD
  ?/.)
(S
  During/IN
  what/WP
  war/NN
  did/VBD
  (NE Modiano/NNP)
  base/NN
  most/JJS
  of/IN
  his/PRP$
  work/NN
  off/IN
  of/IN
  ?/.)
(S
  What/WP
  is/VBZ
  the/DT
  maximum/JJ
  amount/NN
  a/DT
  publishers/NNS
  book/NN
  can/MD
  be/VB
  discounted/VBN
  ?/.)
(S How/WRB many/JJ hospitals/NNS are/VBP in/IN (NE Paris/NNP) ?/.)
(S
  Since/IN
  when/WRB
  have/VBP
  the/DT
  banks/NNS
  of/IN
  the/D

(S
  Who/WP
  became/VBD
  the/DT
  new/JJ
  exclusive/JJ
  carrier/NN
  of/IN
  curling/VBG
  broadcasts/NNS
  during/IN
  the/DT
  2008-2009/CD
  season/NN
  ?/.)
(S
  What/WP
  percent/NN
  of/IN
  the/DT
  previous/JJ
  population/NN
  would/MD
  a/DT
  new/JJ
  digital/JJ
  transmitter/NN
  have/VBP
  served/VBN
  in/IN
  (NE Fredicton/NNP)
  in/IN
  comparison/NN
  to/TO
  the/DT
  analogue/NN
  transmitter/NN
  ?/.)
(S
  When/WRB
  did/VBD
  the/DT
  (NE CBC/NNP)
  announce/NN
  service/NN
  would/MD
  be/VB
  discontinued/VBN
  to/TO
  analogue/VB
  transmitters/NNS
  ?/.)
(S Where/WRB are/VBP (NE HNIC/NNP) broadcasts/NNS aired/VBD ?/.)
(S
  (NE Did/NNP)
  CBC/NNP
  air/NN
  more/RBR
  or/CC
  less/RBR
  live/JJ
  Olympic/NNP
  events/NNS
  than/IN
  (NE NBC/NNP)
  ?/.)
(S
  On/IN
  what/WP
  date/NN
  did/VBD
  (NE CBC/NNP)
  say/VB
  it/PRP
  was/VBD
  striving/VBG
  to/TO
  update/VB
  its/PRP$
  analogue/NN
  transmitters/NNS
  by/IN
  ?/.)
(S
  Which/WDT
  (NE CBC/NNP)
  l

(S
  How/WRB
  many/JJ
  freshman/JJ
  class/NN
  applications/NNS
  did/VBD
  (NE Washington/NNP University/NNP)
  receive/VBP
  in/IN
  2013/CD
  ?/.)
(S
  When/WRB
  were/VBD
  (NE Knight/NNP)
  and/CC
  (NE Bauer/NNP Halls/NNP)
  dedicated/VBN
  ?/.)
(S
  Who/WP
  is/VBZ
  the/DT
  (NE Brown/NNP School/NNP)
  's/POS
  current/JJ
  Dean/NNP
  ?/.)
(S
  When/WRB
  was/VBD
  (NE Washington/NNP University/NNP)
  ranked/VBD
  #/#
  1/CD
  for/IN
  quality/NN
  of/IN
  life/NN
  ?/.)
(S
  How/WRB
  many/JJ
  inventions/NNS
  does/VBZ
  (NE Washington/NNP University/NNP)
  have/VBP
  funded/VBN
  by/IN
  the/DT
  (NE National/NNP Institute/NNP)
  of/IN
  health/NN
  ?/.)
(S
  Who/WP
  is/VBZ
  the/DT
  director/NN
  of/IN
  the/DT
  (NE Genome/NNP Center/NNP)
  at/IN
  (NE Washington/NNP University/NNP)
  ?/.)
(S
  In/IN
  what/WP
  years/NNS
  did/VBD
  the/DT
  (NE Washington/NNP University/NNP)
  men/NNS
  's/POS
  basketball/NN
  teams/NNS
  win/VBP
  division/NN
  championships/NNS
 

(S
  How/WRB
  many/JJ
  media/NNS
  outlets/NNS
  does/VBZ
  (NE Washington/NNP University/NNP)
  support/NN
  ?/.)
(S
  How/WRB
  many/JJ
  research/NN
  centers/NNS
  are/VBP
  housed/VBN
  by/IN
  the/DT
  (NE Brown/NNP School/NNP)
  ?/.)
(S
  Who/WP
  designed/VBD
  the/DT
  new/JJ
  location/NN
  of/IN
  the/DT
  (NE Mildred/NNP Lane/NNP Kemper/NNP Art/NNP Museum/NNP)
  ?/.)
(S
  What/WP
  building/NN
  provides/VBZ
  the/DT
  location/NN
  of/IN
  the/DT
  (NE Washington/NNP University/NNP School/NNP)
  of/IN
  Law/NNP
  ?/.)
(S
  When/WRB
  did/VBD
  (NE Washington/NNP University/NNP)
  begin/VBP
  to/TO
  expand/VB
  west/JJS
  ?/.)
(S
  What/WP
  options/NNS
  are/VBP
  available/JJ
  for/IN
  graduate/NN
  students/NNS
  in/IN
  pursuing/VBG
  an/DT
  MBA/NNP
  at/IN
  (NE Washington/NNP University/NNP)
  ?/.)
(S
  What/WP
  percent/NN
  of/IN
  undergraduate/JJ
  students/NNS
  live/VBP
  on/IN
  campus/NN
  at/IN
  (NE Washington/NNP University/NNP)
  ?/.)
(S
  Who/WP
  is

UnicodeEncodeError: 'ascii' codec can't encode character u'\xe9' in position 5: ordinal not in range(128)

<h3>2. Candidate answering generation</h3>

<h3>3. Candidate answering scoring</h3>

<h3>4. Answer and confidence</h3>